# Pumped-Storage Optimisation with Genetic Algorithm and MILP

In [1]:
import pandas as pd
import numpy as np
import plotnine as pn
import plotly.graph_objs as go
import plotly.express as px
from tqdm.notebook import tqdm
from IPython.display import clear_output, display
import os
from itertools import product

# Import own implementations
from milp import MILP
import genetic
from genetic import GA_Actions_Elite, GA_Actions_Tournament

# Importing tuning libraries
import ray
from ray import train, tune
from ray.tune.search.optuna import OptunaSearch
from ray.tune.schedulers import ASHAScheduler

background_colour = "#F2F2F2"
pn.theme_set(
    pn.theme_classic()
    + pn.theme(
        text=pn.element_text(family="monospace"),
        plot_background=pn.element_rect(
            fill=background_colour, colour=background_colour
        ),
        panel_background=pn.element_rect(
            fill=background_colour, colour=background_colour
        ),
        legend_background=pn.element_rect(
            fill=background_colour, colour=background_colour
        ),
    )
)

%load_ext blackcellmagic

## Reading the Price data

In [2]:
df = pd.read_csv("../01 - Data/example_week.csv")
df.head(2)

,spot,utc_time
0,101.54,2022-01-01 00:00:00+00:00
1,52.13,2022-01-01 01:00:00+00:00


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168 entries, 0 to 167
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   spot      168 non-null    float64
 1   utc_time  168 non-null    object 
dtypes: float64(1), object(1)
memory usage: 2.8+ KB


## The Power Plant

In [4]:
plant_params = {
    "EFFICIENCY": 0.75,
    "MAX_STORAGE_M3": 5000,
    "MIN_STORAGE_M3": 0,
    "TURBINE_POWER_MW": 100,
    "PUMP_POWER_MW": 100,
    "TURBINE_RATE_M3H": 500,
    "MIN_STORAGE_MWH": 0,
    "INITIAL_WATER_LEVEL_PCT": 0,
}
plant_params["INITIAL_WATER_LEVEL"] = (
    plant_params["INITIAL_WATER_LEVEL_PCT"] * plant_params["MAX_STORAGE_M3"]
)
plant_params["PUMP_RATE_M3H"] = (
    plant_params["TURBINE_RATE_M3H"] * plant_params["EFFICIENCY"]
)
plant_params["MAX_STORAGE_MWH"] = (
    plant_params["MAX_STORAGE_M3"] / plant_params["TURBINE_RATE_M3H"]
) * plant_params["TURBINE_POWER_MW"]

## GA Actions

### Elite

In [5]:
ga_solver = GA_Actions_Elite(
    plant_params=plant_params, spot=df["spot"], utc_time=df["utc_time"]
)

In [6]:
analysis = ga_solver.tune(
    tune_config={
        "MUTPB": 1,
        "POP_SIZE": 200,
        "INITIAL_MUTATION_RATE": tune.uniform(0.05, 0.5),
        "FINAL_MUTATION_RATE": tune.uniform(0.005, 0.05),
        "INITIAL_EXPLORATION":tune.uniform(0.1, 0.9),
        "ELITISM": tune.choice(np.arange(0.1, 0.6, 0.1)),
    },
    total_generations=500,
    timeout_s=60*60*4.5,
)

2024-04-09 23:59:14,529	INFO worker.py:1752 -- Started a local Ray instance.
2024-04-09 23:59:14,660	INFO packaging.py:530 -- Creating a file package for local directory '.'.
2024-04-09 23:59:14,676	INFO packaging.py:358 -- Pushing file package 'gcs://_ray_pkg_37a4df1e59311c51.zip' (2.03MiB) to Ray cluster...
2024-04-09 23:59:14,692	INFO packaging.py:371 -- Successfully pushed file package 'gcs://_ray_pkg_37a4df1e59311c51.zip'.
2024-04-09 23:59:16,395	INFO tune.py:613 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
[I 2024-04-09 23:59:16,438] A new study created in memory with name: optuna


Trial name,fitness
train_0025cc65,155509
train_004ee9c8,227828
train_01219c68,224964
train_01766ac7,241941
train_01c84fd8,248310
train_030741c2,218980
train_0572839b,218636
train_06fb3b12,204593
train_0719bad6,-33310.1
train_077aba18,118872


(train pid=8944) C:\arrow\cpp\src\arrow\filesystem\s3fs.cc:2829:  arrow::fs::FinalizeS3 was not called even though S3 was initialized.  This could lead to a segmentation fault at exit
(train pid=27460) C:\arrow\cpp\src\arrow\filesystem\s3fs.cc:2829:  arrow::fs::FinalizeS3 was not called even though S3 was initialized.  This could lead to a segmentation fault at exit
(train pid=26380) C:\arrow\cpp\src\arrow\filesystem\s3fs.cc:2829:  arrow::fs::FinalizeS3 was not called even though S3 was initialized.  This could lead to a segmentation fault at exit
(train pid=28732) C:\arrow\cpp\src\arrow\filesystem\s3fs.cc:2829:  arrow::fs::FinalizeS3 was not called even though S3 was initialized.  This could lead to a segmentation fault at exit
(train pid=10444) C:\arrow\cpp\src\arrow\filesystem\s3fs.cc:2829:  arrow::fs::FinalizeS3 was not called even though S3 was initialized.  This could lead to a segmentation fault at exit
(train pid=12812) C:\arrow\cpp\src\arrow\filesystem\s3fs.cc:2829:  arrow::fs

In [ ]:
analysis.best_config

{'MUTPB': 1,
 'POP_SIZE': 1000,
 'INITIAL_MUTATION_RATE': 0.05824595580106539,
 'FINAL_MUTATION_RATE': 0.005106475700995004,
 'ELITISM': 0.2}

In [ ]:
fig = px.line(
    data_frame=pd.concat(analysis.trial_dataframes.values()),
    x="training_iteration",
    y="fitness",
    color="trial_id",
)
fig.show()

In [ ]:
top_runs = analysis.dataframe().sort_values("fitness", ascending=False).head(10)["trial_id"].to_list()

fig = px.line(
    data_frame=pd.concat(analysis.trial_dataframes.values()).query(
        "trial_id in @top_runs"
    ),
    x="training_iteration",
    y="fitness",
    color="trial_id",
)
fig.show()

In [ ]:
# (
#     analysis.dataframe()
#     .query("trial_id == 'd947e4c7'")
#     .filter(regex="config")
#     .melt()
#     .assign(variable=lambda x: x["variable"].str.replace("config/", ""))
#     .set_index("variable")
#     .to_dict()["value"]
# )

### Tournament

In [ ]:
ga_solver = GA_Actions_Tournament(
    plant_params=plant_params, spot=df["spot"], utc_time=df["utc_time"]
)

In [ ]:
analysis = ga_solver.tune(
    tune_config={
        "MUTPB": 1,
        "POP_SIZE": 200,
        "CXPB": tune.uniform(0.05, 0.95),
        "INITIAL_MUTATION_RATE": tune.uniform(0.05, 0.5),
        "FINAL_MUTATION_RATE": tune.uniform(0.005, 0.05),
        "INITIAL_EXPLORATION":tune.uniform(0.1, 0.9),
        "TOURNAMENT_SIZE": tune.randint(1, 100),
    },
    total_generations=500,
    timeout_s=60*60*4.5,
)

2024-04-02 03:18:27,177	INFO worker.py:1585 -- Calling ray.init() again after it has already been called.
2024-04-02 03:18:27,182	INFO tune.py:613 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
[I 2024-04-02 03:18:27,190] A new study created in memory with name: optuna


Trial name,fitness
train_080d885f,-7.91266e+06
train_08245116,-7.46613e+06
train_0ada0742,-7.33731e+06
train_1c766c5e,-7.32107e+06
train_1ef91201,-7.32713e+06
train_20276f3b,-7.83793e+06
train_24f94535,-7.81118e+06
train_2c8c313d,-7.36702e+06
train_30948727,-8.58044e+06
train_3cb79978,-8.68326e+06


2024-04-02 03:32:44,959	WARNING util.py:202 -- The `on_step_begin` operation took 1.637 s, which may be a performance bottleneck.
2024-04-02 03:36:17,957	WARNING util.py:202 -- The `on_step_begin` operation took 0.882 s, which may be a performance bottleneck.
2024-04-02 03:38:19,537	WARNING util.py:202 -- The `on_step_begin` operation took 0.739 s, which may be a performance bottleneck.
2024-04-02 03:40:21,816	WARNING util.py:202 -- The `on_step_begin` operation took 1.541 s, which may be a performance bottleneck.
2024-04-02 03:44:57,099	WARNING util.py:202 -- The `on_step_begin` operation took 3.137 s, which may be a performance bottleneck.
2024-04-02 03:46:58,831	WARNING util.py:202 -- The `on_step_begin` operation took 1.093 s, which may be a performance bottleneck.
2024-04-02 04:01:38,627	WARNING util.py:202 -- The `on_step_begin` operation took 2.104 s, which may be a performance bottleneck.
2024-04-02 04:02:19,422	WARNING util.py:202 -- The `on_step_begin` operation took 0.666 s,

In [ ]:
analysis.best_config

{'MUTPB': 1,
 'POP_SIZE': 10000,
 'CXPB': 0.8541176958467338,
 'INITIAL_MUTATION_RATE': 0.19576303144411172,
 'FINAL_MUTATION_RATE': 0.005366961802895822,
 'TOURNAMENT_SIZE': 72}

In [ ]:
fig = px.line(
    data_frame=pd.concat(analysis.trial_dataframes.values()),
    x="training_iteration",
    y="fitness",
    color="trial_id",
)
fig.show()

In [ ]:
top_runs = analysis.dataframe().sort_values("fitness", ascending=False).head(10)["trial_id"].to_list()

fig = px.line(
    data_frame=pd.concat(analysis.trial_dataframes.values()).query(
        "trial_id in @top_runs"
    ),
    x="training_iteration",
    y="fitness",
    color="trial_id",
)
fig.show()

In [ ]:
(
    analysis.dataframe()
    .query("trial_id == 'dd531b71'")
    .filter(regex="config")
    .melt()
    .assign(variable=lambda x: x["variable"].str.replace("config/", ""))
    .set_index("variable")
    .to_dict()["value"]
)

{'MUTPB': 1.0,
 'POP_SIZE': 10000.0,
 'CXPB': 0.8541176958467338,
 'INITIAL_MUTATION_RATE': 0.19576303144411172,
 'FINAL_MUTATION_RATE': 0.005366961802895822,
 'TOURNAMENT_SIZE': 72.0}